In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pestools as pt
import pyemu
import os, shutil
from matplotlib.backends.backend_pdf import PdfPages

# Looking at sensitivity

### We have already discussed the Jacobian matrix in a few places. Let's take a look at it more closely and see what we can learn from it and how to handle such information as the number of parameters rises.

### Let's pull in some spatial data first for plotting when we look at pilot points and make a couple plotting functions

In [ ]:
# observation locations
obslox = pd.read_csv('freyberg.hyd', delim_whitespace=True, usecols = [4,5,6], 
                     index_col=2, skiprows = 1, header=None, names=['X','Y','obsname'])
# parameter locations
parlox = pd.read_csv('points1.dat.tpl', delim_whitespace=True, usecols=[0,1,2],
                    index_col=0, skiprows=1, header=None, names=['parname','X','Y'])

In [ ]:
def plot_Jacobian(jac, figsize, cmap='viridis',logtrans=True):
    f = plt.figure(figsize=figsize)
    ax = plt.axes([0, 0.05, 0.9, 0.9 ]) #left, bottom, width, height
    if logtrans:
        jcdata=np.log(np.abs(jac.df()))
    else:
        jcdata=jac.df()
    im = ax.imshow(jcdata, interpolation='nearest', cmap=cmap, aspect='auto')
    plt.xticks(range(len(jac.col_names)), jac.col_names, rotation=90)
    plt.yticks(range(len(jac.row_names)), jac.row_names)
    
    ax.grid(False)
    cax = plt.axes([0.95, 0.05, 0.05,0.9 ])
    plt.colorbar(mappable=im, cax=cax)

In [ ]:
def spatial_plot_sens(jac,cobs, obslox, parlox, figsize=(4,7)):
    sens = jac.df().loc[cobs]
    sens.drop('rch1',inplace=1)
    fig=plt.figure(figsize=figsize)
    plt.plot(parlox.X,parlox.Y,'kd',markersize=.8)
    scalefactor=5
    if 'flux' not in cobs:
        coblox = obslox.loc[cobs]
        plt.plot(coblox.X,coblox.Y,'kx', markersize=10)
        scalefactor=1000    
    plt.scatter(parlox.X,parlox.Y, s=np.abs(sens.values)*scalefactor, c=sens.values, cmap='viridis')
    plt.axis('equal')
    plt.colorbar()
    plt.title('Sensitivity for {0}'.format(cobs))
    plt.xlim(0,5000)
    plt.ylim(0,10000)
    plt.axis('off')
    return fig

### Look at the Jacobian matrix---gradients of parameters wrt. observations

For each parameter-observation combination, we can see how much the observation value changes due to a small change in the parameter. If $y$ are the observations and $x$ are the parameters, the equation for the $i^th$ observation with respect to the $j^th$ parameter is:  
## $\frac{\partial y_i}{\partial x_j}$
This can be approximated by finite differences as :  
## $\frac{\partial y_i}{\partial x_j}~\frac{y\left(x+\Delta x \right)-y\left(x\right)}{\Delta x}$

### First we can read in a couple Jacobian matrices -- one from our simple model, and one from a more complex one

In [ ]:
jac_simple = pyemu.Jco.from_binary(os.path.join('..','..','models','Freyberg','Freyberg_K_and_R','freyberg.jcb'))
jac_complex = pyemu.Jco.from_binary(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))

### These are now matrices. How big are they?

In [ ]:
print ('simple  --> {0} rows x {1} columns'.format(*jac_simple.shape))
print ('complex --> {0} rows x {1} columns'.format(*jac_complex.shape))


In [ ]:
plot_Jacobian(jac_simple, figsize=(7,4))

In [ ]:
# Let's drop all the forecasts and regularization information
jac_simple.drop([x for x in jac_simple.df().index if x.startswith('fr')], axis=0)
jac_simple.drop([x for x in jac_simple.df().index if 'fore' in x], axis=0)
jac_simple.drop('travel_time', axis=0)

In [ ]:
plot_Jacobian(jac_simple, figsize=(7,4))

In [ ]:
plot_Jacobian(jac_complex, figsize=(7,4))

In [ ]:
# Let's drop all the forecasts and regularization information
jac_complex.drop([x for x in jac_complex.df().index if x.startswith('i')], axis=0)
jac_complex.drop([x for x in jac_complex.df().index if x.startswith('fr')], axis=0)
jac_complex.drop([x for x in jac_complex.df().index if 'fore' in x], axis=0)
jac_complex.drop('travel_time', axis=0)

In [ ]:
plot_Jacobian(jac_complex, figsize=(7,4))

# Can be more informative to look at sensitivity spatially

In [ ]:
print(jac_complex.row_names)

In [ ]:
spatial_plot_sens(jac_complex,'rivflux_cal', obslox,parlox);

In [ ]:
with PdfPages('allsens.pdf') as ofp:
    for cob in jac_complex.row_names:
        cf = spatial_plot_sens(jac_complex, cob, obslox,parlox)
        ofp.savefig()
        plt.close('all')

# How about Composite Scaled Sensitivities
In the traditional, overdetermined regression world, CSS was a popular metric. CSS is Composite Scaled Sensitivitity.

In Hill and Tiedeman (2007) this is calculated as: 
## ${css_{j}=\sqrt{\left(\sum_{i-1}^{ND}\left(\frac{\partial y'_{i}}{\partial b_{j}}\right)\left|b_{j}\right|\sqrt{w_{ii}}\right)/ND}}$

In PEST, Doherty calculates it slightly differently in that scaling by the parameter values happens automatically when the parameter is subjected to a log-transform. This is due to a correction that must be made in calculating the Jacobian matrix and follows from the chain rule of derivatives.


In [ ]:
la = pyemu.LinearAnalysis(jco=os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))

In [ ]:
plt.figure(figsize=(8,4))
ax = la.get_par_css_dataframe()['pest_css'].sort_values(ascending=False).plot(kind='bar')
ax.set_yscale('log')

### Now let's consider correlation and posterior covariance

In [ ]:
sc = pyemu.Schur(os.path.join('..','..','models','Freyberg','Freyberg_pilotpoints','freyberg_pp_reg_phimlim26.jcb'))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df().head()

In [ ]:
R = covar.to_pearson()
plt.imshow(R.df(), interpolation='nearest', cmap='viridis')
plt.colorbar()

In [ ]:
cpar = 'hkpp10'
R.df().loc[cpar][np.abs(R.df().loc[cpar])>.5]

In [ ]:
R_plot = R.df().as_matrix()
R_plot[np.abs(R_plot)>.9] = np.nan
plt.imshow(R_plot, interpolation='nearest', cmap='viridis')
plt.colorbar()